# Similarity Search with time-based filtering using LangChain

A key use case for Timescale Vector is efficient time-based vector search. Timescale Vector enables this by automatically partitioning vectors (and associated metadata) by time. This allows you to efficiently query vectors by both similarity to a query vector and time because we can exclude entire partitions that don't overlap with the query time.

Time-based vector search functionality is helpful for applications like:
- Storing and retrieving LLM response history (e.g. chatbots)
- Finding the most recent embeddings that are similar to a query vector (e.g recent news).
- Constraining similarity search to a relevant time range (e.g asking time-based questions about a knowledge base)

To illustrate how to use TimescaleVector's time-based vector search functionality, we'll ask questions about the git log history for TimescaleDB . We'll illustrate how to add documents with a time-based uuid and how run similarity searches with time range filters.

## Setup your environment

First, install the necessary prerequisites

In [ ]:
# Pip install necessary packages
%pip install timescale-vector
%pip install openai
%pip install langchain
%pip install python-dotenv

Next, setup your secrets. You'll need an API key from [OpenAI](https://platform.openai.com/) and a service url from [Timescale](https://console.cloud.timescale.com/signup?utm_campaign=vectorlaunch&utm_source=aicookbooks&utm_medium=referral). For security, we suggest storing these in a dotenv file.

In [ ]:
import os

# Get openAI api key by reading local .env file
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv(), override=True)
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
TIMESCALE_SERVICE_URL = os.environ["TIMESCALE_SERVICE_URL"]

## Extract content and metadata from git log JSON
First lets load in the git log data into a new collection in our PostgreSQL database named `timescale_commits`

You'll need to [download the sample dataset](https://s3.amazonaws.com/assets.timescale.com/ai/ts_git_log.json) and place it in the same directory as this notebook.

You can use following command:

In [ ]:
# Download the file using curl and save it as commit_history.csv
# Note: Execute this command in your terminal, in the same directory as the notebook
!curl -O "https://s3.amazonaws.com/assets.timescale.com/ai/ts_git_log.json"

Next, let's create a set of functions to extract metadata from the git commits. Note, how we create the UUID based on the timestamp.

In [ ]:
from timescale_vector import client
from datetime import datetime

def parse_date(date_string: str) -> datetime:
    if date_string is None:
        return None
    time_format = "%a %b %d %H:%M:%S %Y %z"
    return datetime.strptime(date_string, time_format)

def split_name(input_string: str):
    if input_string is None:
        return None, None
    start = input_string.find("<")
    end = input_string.find(">")
    name = input_string[:start].strip()
    email = input_string[start + 1 : end].strip()
    return name, email

def extract_metadata(record: dict, metadata: dict) -> dict:
    dt = parse_date(record["date"])
    record_name, record_email = split_name(record["author"])
    # it is important to use a uuid v1 that contains the timestamp
    metadata["id"] = str(client.uuid_from_time(dt))
    if dt is not None:
        metadata["date"] = dt.isoformat()
    else:
        metadata["date"] = None
    metadata["author"] = record["author"]
    metadata["author_name"] = record_name
    metadata["author_email"] = record_email
    metadata["commit_hash"] = record["commit"]
    return metadata

Then, we loade the data.

In [ ]:
from langchain.document_loaders.json_loader import JSONLoader

# Define path to the JSON file relative to this notebook
# Change this to the path to your JSON file
FILE_PATH = "ts_git_log.json"

# Load data from JSON file and extract metadata
loader = JSONLoader(
    file_path=FILE_PATH,
    jq_schema=".commit_history[]",
    text_content=False,
    metadata_func=extract_metadata,
)
documents = loader.load()

# Remove documents with None dates
documents = [doc for doc in documents if doc.metadata["date"] is not None]

Since this is a demo, we will only load the first 500 records. In practice, you can load as many records as you want.

We use the CharacterTextSplitter to split the documents into smaller chunks if needed for easier embedding. Note that this splitting process retains the metadata for each document.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

NUM_RECORDS = 500
documents = documents[:NUM_RECORDS]

# Split the documents into chunks for embedding
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
docs = text_splitter.split_documents(documents)

Let's see one document:

In [ ]:
print(documents[0])

## Load documents and metadata into TimescaleVector vectorstore
Now that we have prepared our documents, let's process them and load them, along with their vector embedding representations into our Timescale Vector.

First, we'll define a collection name, which will be the name of our table in the PostgreSQL database. 

We'll also define a time delta, which we pass to the `time_partition_interval` argument, which will be used to as the interval for partitioning the data by time. Each partition will consist of data for the specified length of time. We'll use 7 days for simplicity, but you can pick whatever value make sense for your use case -- for example if you query recent vectors frequently you might want to use a smaller time delta like 1 day, or if you query vectors over a decade long time period then you might want to use a larger time delta like 6 months or 1 year.

Finally, we'll create the TimescaleVector instance. We specify the `ids` argument to be the `uuid` field in our metadata that we created in the pre-processing step above. We do this because we want the time part of our uuids to reflect dates in the past (i.e when the commit was made). However, if we wanted the current date and time to be associated with our document, we can remove the id argument and uuid's will be automatically created with the current date and time.

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.timescalevector import TimescaleVector
from datetime import timedelta

# Define collection name
COLLECTION_NAME = "timescale_commits"
embeddings = OpenAIEmbeddings()

# Create a Timescale Vector instance from the collection of documents
db = TimescaleVector.from_documents(
    embedding=embeddings,
    ids=[doc.metadata["id"] for doc in docs],
    documents=docs,
    collection_name=COLLECTION_NAME,
    service_url=TIMESCALE_SERVICE_URL,
    time_partition_interval=timedelta(days=7),
)

Create an index to speed up queries. Using the `create_index()` function without additional arguments will create a timescale_vector_index by default, using the default parameters.

In [ ]:
# create an index
# by default this will create a Timescale Vector (DiskANN) index
db.create_index()

## Querying vectors by time and similarity

Now that we have loaded our documents into TimescaleVector, we can query them by time and similarity.

TimescaleVector does this effeciently because any partition (7-day period in this example) that does not overlap with the query is completely excluded.

TimescaleVector provides multiple methods for querying vectors by doing similarity search with time-based filtering. 

Let's take a look at each method below:

In [ ]:
# Time filter variables
start_dt = datetime(2023, 8, 1, 22, 10, 35)  # Start date = 1 August 2023, 22:10:35
end_dt = datetime(2023, 8, 30, 22, 10, 35)  # End date = 30 August 2023, 22:10:35
td = timedelta(days=7)  # Time delta = 7 days

query = "What's new with TimescaleDB functions?"

Method 1: Filter within a provided start date and end date.

In [ ]:
# Method 1: Query for vectors between start_date and end_date
docs_with_score = db.similarity_search_with_score(
    query, start_date=start_dt, end_date=end_dt
)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print("Date: ", doc.metadata["date"])
    print(doc.page_content)
    print("-" * 80)

Method 2: Filter within a provided start date, and a time delta later.

In [ ]:
# Method 2: Query for vectors between start_dt and a time delta td later
# Most relevant vectors between 1 August and 7 days later
docs_with_score = db.similarity_search_with_score(
    query, start_date=start_dt, time_delta=td
)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print("Date: ", doc.metadata["date"])
    print(doc.page_content)
    print("-" * 80)

Method 3: Filter within a provided end date and a time delta earlier.

In [ ]:
# Method 3: Query for vectors between end_dt and a time delta td earlier
# Most relevant vectors between 30 August and 7 days earlier
docs_with_score = db.similarity_search_with_score(query, end_date=end_dt, time_delta=td)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print("Date: ", doc.metadata["date"])
    print(doc.page_content)
    print("-" * 80)

## Querying using other metadata.

You can also filter based on other metadata.  This shows a similarity search with a filter on author name.

In [ ]:
docs_with_score = db.similarity_search_with_score(
    query,filter={"author_name": "Sven Klemm"}
)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print("Date: ", doc.metadata["date"])
    print(doc.page_content)
    print("-" * 80)

You can also combine it with time-based search. This shows a similarity search with a time filter as well as a filter on author name.

In [ ]:
docs_with_score = db.similarity_search_with_score(
    query, start_date=start_dt, end_date=end_dt, filter={"author_name": "Sven Klemm"}
)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print("Date: ", doc.metadata["date"])
    print(doc.page_content)
    print("-" * 80)